In [1]:
from tensorflow import keras
from keras.datasets import imdb

In [2]:
# 가장 자주 나타나는 단어 300개만 사용
(train_data, train_target),(test_data, test_target) = imdb.load_data(num_words=300)

17473536/17464789 [==============================] - 1s 0us/step


In [3]:
print(train_data.shape, train_target.shape)
print(test_data.shape, test_target.shape)

(25000,) (25000,)
(25000,) (25000,)


In [4]:
from sklearn.model_selection import train_test_split
train_data, val_data, train_target, val_target = train_test_split(train_data, train_target,
                                                                  test_size=0.2, random_state=0)

In [5]:
# 리뷰 길이를 100으로 맞추기
from keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_data, maxlen=100)
print(train_seq.shape)

(20000, 100)


In [6]:
val_seq = pad_sequences(val_data, maxlen=100)
print(val_seq.shape)

(5000, 100)


In [7]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Embedding(300, 5, input_length=100))
model.add(layers.LSTM(8))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 5)            1500      
                                                                 
 lstm (LSTM)                 (None, 8)                 448       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 1,957
Trainable params: 1,957
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-lstm.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(train_seq, train_target, epochs=100,
                    validation_data=(val_seq, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
625/625 [==============================] - 12s 8ms/step - loss: 0.6206 - accuracy: 0.6449 - val_loss: 0.5355 - val_accuracy: 0.7372
Epoch 2/100
625/625 [==============================] - 5s 8ms/step - loss: 0.5141 - accuracy: 0.7546 - val_loss: 0.5147 - val_accuracy: 0.7468
Epoch 3/100
625/625 [==============================] - 4s 7ms/step - loss: 0.4982 - accuracy: 0.7620 - val_loss: 0.5536 - val_accuracy: 0.7162
Epoch 4/100
625/625 [==============================] - 5s 7ms/step - loss: 0.4949 - accuracy: 0.7646 - val_loss: 0.5123 - val_accuracy: 0.7462
Epoch 5/100
625/625 [==============================] - 5s 7ms/step - loss: 0.4913 - accuracy: 0.7660 - val_loss: 0.5037 - val_accuracy: 0.7496
Epoch 6/100
625/625 [==============================] - 5s 7ms/step - loss: 0.4876 - accuracy: 0.7666 - val_loss: 0.5213 - val_accuracy: 0.7416
Epoch 7/100
625/625 [==============================] - 4s 7ms/step - loss: 0.4850 - accuracy: 0.7685 - val_loss: 0.4989 - val_accuracy: 0.747

In [10]:
test_seq = pad_sequences(test_data, maxlen=100)
model.evaluate(test_seq[:100], test_target[:100])

4/4 [==============================] - 0s 5ms/step - loss: 0.4814 - accuracy: 0.7900


[0.4814051687717438, 0.7900000214576721]

In [11]:
# Multi-Layer LSTM
model2 = models.Sequential()
model2.add(layers.Embedding(300, 5, input_length=100))
model2.add(layers.LSTM(8, return_sequences=True))
model2.add(layers.LSTM(8))
model2.add(layers.Dense(1, activation='sigmoid'))

In [12]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 5)            1500      
                                                                 
 lstm_1 (LSTM)               (None, 100, 8)            448       
                                                                 
 lstm_2 (LSTM)               (None, 8)                 544       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 2,501
Trainable params: 2,501
Non-trainable params: 0
_________________________________________________________________


In [13]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model2.fit(train_seq, train_target, epochs=100,
                     validation_data=(val_seq, val_target),
                     callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
625/625 [==============================] - 11s 13ms/step - loss: 0.5870 - accuracy: 0.6720 - val_loss: 0.5232 - val_accuracy: 0.7376
Epoch 2/100
625/625 [==============================] - 7s 11ms/step - loss: 0.5091 - accuracy: 0.7560 - val_loss: 0.5374 - val_accuracy: 0.7288
Epoch 3/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4975 - accuracy: 0.7612 - val_loss: 0.5078 - val_accuracy: 0.7492
Epoch 4/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4935 - accuracy: 0.7646 - val_loss: 0.5071 - val_accuracy: 0.7534
Epoch 5/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4864 - accuracy: 0.7648 - val_loss: 0.4999 - val_accuracy: 0.7562
Epoch 6/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4838 - accuracy: 0.7710 - val_loss: 0.5045 - val_accuracy: 0.7500
Epoch 7/100
625/625 [==============================] - 7s 12ms/step - loss: 0.4812 - accuracy: 0.7695 - val_loss: 0.5004 - val_accuracy

In [14]:
model2.evaluate(test_seq[:100], test_target[:100])

4/4 [==============================] - 0s 8ms/step - loss: 0.4881 - accuracy: 0.7600


[0.488075852394104, 0.7599999904632568]